In [1]:
import pandas as pd
import os

df = pd.read_csv('LostIslandBuildings.csv')

#Rename Color / Area column to be Color
df.rename(columns={'Color / Area': 'Color'}, inplace=True)

#Separate Coordinates into X-Coord and Y-Coord
df[['X-Coord', 'Y-Coord']] = df['Coordinates'].str.split(' ', expand=True)
df = df.drop('Coordinates', axis=1)
df['X-Coord'] = df['X-Coord'].str[2:]
df['Y-Coord'] = df['Y-Coord'].str[2:]

#Separate Buff Percent into categories
buffCategories = ['HP Guardian', 'HP Shooter', 'HP Carrier', 'ATK Guardian', 'ATK Shooter', 'ATK Carrier', 'DEF Guardian', 'DEF Shooter', 'DEF Carrier']

for i in range(len(buffCategories)):
    df[buffCategories[i]] = df['Buff Percent'].apply(lambda x: x.split(' ')[0] if buffCategories[i] in x else 0)

#Separate Buff Percent into categories for Wonders
wonderCondition = df['Alliance Influence'] == '25,000'
wonderRowsOnly = df.loc[wonderCondition, :]
wonderRowBuffs = ['3.6%', '3.6%', '3.6%', '9%', '9%', '9%', '9%', '9%', '9%']
for i in range(len(buffCategories)):
    df.loc[wonderCondition, buffCategories[i]] = wonderRowBuffs[i]
df.loc[wonderCondition, buffCategories] = wonderRowsOnly[buffCategories]

df = df.drop('Buff Percent', axis=1)

#Separate Soldier Production into categories
productionCategories = ['Guardian', 'Shooter', 'Carrier']

for i in range(len(productionCategories)):
    df[productionCategories[i] + ' Output'] = df['Soldier Production'].apply(lambda x: x.split(' ')[0] if productionCategories[i] in x else 0)

#Separate Soldier Production for Wonders
wonderRowProduction = ['44/h', '44/h', '44/h']
productionCategoriesRenamed = ['Guardian Output', 'Shooter Output', 'Carrier Output']
for i in range(len(productionCategoriesRenamed)):
    wonderRowsOnly[productionCategoriesRenamed[i]] = wonderRowProduction[i]
df.loc[wonderCondition, productionCategoriesRenamed] = wonderRowsOnly[productionCategoriesRenamed]

df = df.drop('Soldier Production', axis=1)

#Rename Buildings to Building
df.rename(columns={'Buildings': 'Building'}, inplace=True)

#Formatting
df = df.apply(lambda x: x.str.replace('%', '', regex=True))
df = df.apply(lambda x: x.str.replace('/h', '', regex=True))
df = df.apply(lambda x: x.str.replace(',', '', regex=True))
df = df.fillna(0)

df['Alliance Influence'] = df['Alliance Influence'].astype('int')
for i in range(len(buffCategories)):
    df[buffCategories[i]] = df[buffCategories[i]].astype(float)
for i in range(len(productionCategoriesRenamed)):
    df[productionCategoriesRenamed[i]] = df[productionCategoriesRenamed[i]].astype('int')

df.to_csv('UpdatedLostIslandBuildings.csv', index=False)

In [85]:
#Convert csv file to SQLite database
import sqlite3 as sq3

updatedDf = pd.read_csv('UpdatedLostIslandBuildings.csv')
conn = sq3.connect('LostIslandBuildings.db')
updatedDf.to_sql('Buildings', conn, if_exists='replace', index=False)
conn.close()